Following Script:
* Crawls the given folder 
* Extract cluster information from each file
* Picks the revelent clusters
* executes FSL's queryatlas to find the details of the region

In [1]:
!atlasquery -a "Harvard-Oxford Cortical Structural Atlas" -c 30,-42,72 
!atlasquery -a "Harvard-Oxford Subcortical Structural Atlas" -c 30,30,30 
!atlasquery -a  "Juelich Histological Atlas" -c 30,30,30 


#--dumpatlases "Harvard-Oxford Subcortical Structural Atlas"

<b>Harvard-Oxford Cortical Structural Atlas</b><br>33% Superior Parietal Lobule, 9% Postcentral Gyrus
<b>Harvard-Oxford Subcortical Structural Atlas</b><br>63% Right Cerebral White Matter, 36% Right Cerebral Cortex 
<b>Juelich Histological Atlas</b><br>No label found!


In [2]:
import subprocess

p = subprocess.Popen(["atlasquery", "-a", "Harvard-Oxford Cortical Structural Atlas", "-c", "30,30,30" ], stdout=subprocess.PIPE)

out = p.communicate()

str(out[0]).split('>')[1], str(out[0]).split('>')[3]

('Harvard-Oxford Cortical Structural Atlas</b',
 "27% Middle Frontal Gyrus, 2% Frontal Pole\\n'")

In [3]:
out

(b'<b>Harvard-Oxford Cortical Structural Atlas</b><br>27% Middle Frontal Gyrus, 2% Frontal Pole\n',
 None)

# Ok Lets Start the code

In [1]:
import glob
import os
from os.path import join as opj
import re
import pandas as pd
import numpy  as np

thresh = 100
folder = 'clustersHypoConnected'
# folder = 'clustersHyperConnected'

name_csv = folder + '_' +  str(thresh) + '.csv'

lst = glob.glob(opj(folder,'*.txt'))




In [ ]:
%%time
table = pd.DataFrame()
for path in lst:
    z = re.match(".*Roi(\d{4})_.",path)
    roi = z.group(1)

    
    roi = int(roi)+1
    print('ROI Number ',roi)
    df = pd.read_table(path)

    df_selected = df.loc[(df['Voxels'] > thresh)]
    df_peak_tval_coordinates = df_selected.as_matrix(['Voxels','MAX','MAX X (mm)' ,'MAX Y (mm)','MAX Z (mm)'])
    
    if len(df_peak_tval_coordinates) > 0:
        print(df_peak_tval_coordinates)
        
        for row in df_peak_tval_coordinates:
            print(row[2],row[3],row[4])
            coord = str(row[2])+ ','+ str(row[3]) + ',' + str(row[4])
            p1 = subprocess.Popen(["atlasquery", "-a", "Harvard-Oxford Cortical Structural Atlas", "-c", coord ], stdout=subprocess.PIPE)
            p2 = subprocess.Popen(["atlasquery", "-a", "Harvard-Oxford Subcortical Structural Atlas", "-c", coord], stdout=subprocess.PIPE)
            p3 = subprocess.Popen(["atlasquery", "-a", "Juelich Histological Atlas", "-c",  coord], stdout=subprocess.PIPE)

            out1 = p1.communicate()
            out2 = p2.communicate()
            out3 = p3.communicate()
            
            d = {'ROI':[roi], 'Voxels':[row[0]], 'Max_tvalue':[row[1]], 'MNI(X Y Z)':[coord], 'Harvard-Oxford Cortical Structural Atlas':[str(out1[0]).split('>')[3]], 'Harvard-Oxford Subcortical Structural Atlas':str(out2[0]).split('>')[3], 'Juelich Histological Atlas':str(out3[0]).split('>')[3]}
#             var = pd.DataFrame([roi, row[0], row[1], coord, str(out1[0]).split('>')[3], \
#                   str(out2[0]).split('>')[3], \
#                   str(out3[0]).split('>')[3]], columns=['ROI', 'Voxels', 'Max_tvalue', 'MNI(X Y Z)', 'Harvard-Oxford Cortical Structural Atlas', 'Harvard-Oxford Subcortical Structural Atlas', 'Juelich Histological Atlas'])  
            var = pd.DataFrame(d)
            print("var:",var)
            
#             var.columns = ['ROI', 'Voxels', 'Max_tvalue', 'MNI(X Y Z)', 'Harvard-Oxford Cortical Structural Atlas', 'Harvard-Oxford Subcortical Structural Atlas', 'Juelich Histological Atlas']
            table = table.append(var)[['ROI', 'Voxels', 'Max_tvalue', 'MNI(X Y Z)', 'Harvard-Oxford Cortical Structural Atlas', 'Harvard-Oxford Subcortical Structural Atlas', 'Juelich Histological Atlas']]
            
#             [var.columns.tolist()]
            
table.to_csv(name_csv)

ROI Number  1
ROI Number  2
ROI Number  3
[[ 418.      5.09  -68.    -36.     -4.  ]]
-68.0 -36.0 -4.0
var:             Harvard-Oxford Cortical Structural Atlas  \
0  70% Middle Temporal Gyrus, posterior division,...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  83% Left Cerebral Cortex , 1% Left Cerebral Wh...   

  Juelich Histological Atlas        MNI(X Y Z)  Max_tvalue  ROI  Voxels  
0         No label found!\n'  -68.0,-36.0,-4.0        5.09    3   418.0  
ROI Number  4
ROI Number  5
[[ 112.      4.73  -62.    -38.     -6.  ]]
-62.0 -38.0 -6.0
var:             Harvard-Oxford Cortical Structural Atlas  \
0  50% Middle Temporal Gyrus, posterior division,...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  62% Left Cerebral Cortex , 38% Left Cerebral W...   

  Juelich Histological Atlas        MNI(X Y Z)  Max_tvalue  ROI  Voxels  
0         No label found!\n'  -62.0,-38.0,-6.0        4.73    5   112.0  
ROI Number  6
[[ 658.      5.3   -66.    -24.    -

var:             Harvard-Oxford Cortical Structural Atlas  \
0  24% Temporal Occipital Fusiform Cortex, 12% Te...   

  Harvard-Oxford Subcortical Structural Atlas Juelich Histological Atlas  \
0               41% Right Cerebral Cortex \n'         No label found!\n'   

         MNI(X Y Z)  Max_tvalue  ROI  Voxels  
0  36.0,-42.0,-26.0        4.72   13   404.0  
-6.0 -24.0 44.0
var:             Harvard-Oxford Cortical Structural Atlas  \
0  58% Cingulate Gyrus, posterior division, 22% P...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  90% Left Cerebral Cortex , 10% Left Cerebral W...   

                          Juelich Histological Atlas       MNI(X Y Z)  \
0  18% GM Primary motor cortex BA4a L, 10% GM Pre...  -6.0,-24.0,44.0   

   Max_tvalue  ROI  Voxels  
0        4.32   13   257.0  
-2.0 54.0 0.0
var:             Harvard-Oxford Cortical Structural Atlas  \
0  50% Paracingulate Gyrus, 18% Frontal Pole, 14%...   

         Harvard-Oxford Subcortical Structural Atla

var:             Harvard-Oxford Cortical Structural Atlas  \
0  33% Superior Parietal Lobule, 9% Postcentral G...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  49% Right Cerebral Cortex , 1% Right Cerebral ...   

                          Juelich Histological Atlas       MNI(X Y Z)  \
0  60% GM Primary somatosensory cortex BA1 R, 19%...  30.0,-42.0,72.0   

   Max_tvalue  ROI  Voxels  
0        4.42   20   545.0  
8.0 -30.0 42.0
var:             Harvard-Oxford Cortical Structural Atlas  \
0  65% Cingulate Gyrus, posterior division, 8% Pr...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  79% Right Cerebral Cortex , 21% Right Cerebral...   

                          Juelich Histological Atlas      MNI(X Y Z)  \
0  43% GM Superior parietal lobule 5Ci R, 20% GM ...  8.0,-30.0,42.0   

   Max_tvalue  ROI  Voxels  
0        4.19   20   435.0  
-48.0 -10.0 56.0
var:          Harvard-Oxford Cortical Structural Atlas  \
0  51% Precentral Gyrus, 12% Postcentral

var:             Harvard-Oxford Cortical Structural Atlas  \
0  42% Middle Temporal Gyrus, posterior division,...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  52% Right Cerebral White Matter, 48% Right Cer...   

  Juelich Histological Atlas        MNI(X Y Z)  Max_tvalue  ROI  Voxels  
0         No label found!\n'  52.0,-26.0,-10.0        4.39   43   226.0  
-48.0 -62.0 42.0
var:             Harvard-Oxford Cortical Structural Atlas  \
0  56% Lateral Occipital Cortex, superior divisio...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  84% Left Cerebral Cortex , 16% Left Cerebral W...   

                          Juelich Histological Atlas        MNI(X Y Z)  \
0  73% GM Inferior parietal lobule Pga L, 44% GM ...  -48.0,-62.0,42.0   

   Max_tvalue  ROI  Voxels  
0        3.93   43   116.0  
60.0 -60.0 20.0
var:             Harvard-Oxford Cortical Structural Atlas  \
0  31% Lateral Occipital Cortex, superior divisio...   

         Harvard-Oxford Subcorti

var:             Harvard-Oxford Cortical Structural Atlas  \
0  66% Inferior Temporal Gyrus, temporooccipital ...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  85% Right Cerebral Cortex , 13% Right Cerebral...   

  Juelich Histological Atlas        MNI(X Y Z)  Max_tvalue  ROI  Voxels  
0         No label found!\n'  50.0,-54.0,-18.0         5.5   55  6505.0  
14.0 -88.0 36.0
var:             Harvard-Oxford Cortical Structural Atlas  \
0  38% Occipital Pole, 21% Lateral Occipital Cort...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  67% Right Cerebral Cortex , 22% Right Cerebral...   

          Juelich Histological Atlas       MNI(X Y Z)  Max_tvalue  ROI  Voxels  
0  10% GM Visual cortex V2 BA18 R\n'  14.0,-88.0,36.0        4.08   55  2435.0  
-24.0 -84.0 36.0
var:             Harvard-Oxford Cortical Structural Atlas  \
0  62% Lateral Occipital Cortex, superior divisio...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  68% Left Cerebral 

var:             Harvard-Oxford Cortical Structural Atlas  \
0  48% Middle Temporal Gyrus, posterior division,...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  90% Left Cerebral Cortex , 9% Left Cerebral Wh...   

  Juelich Histological Atlas         MNI(X Y Z)  Max_tvalue  ROI  Voxels  
0         No label found!\n'  -64.0,-12.0,-16.0        4.48   42   570.0  
42.0 -54.0 24.0
var:             Harvard-Oxford Cortical Structural Atlas  \
0  26% Angular Gyrus, 1% Lateral Occipital Cortex...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  71% Right Cerebral White Matter, 29% Right Cer...   

                          Juelich Histological Atlas       MNI(X Y Z)  \
0  27% GM Inferior parietal lobule Pga R, 10% GM ...  42.0,-54.0,24.0   

   Max_tvalue  ROI  Voxels  
0        3.93   42   154.0  
ROI Number  56
[[  5.37000000e+03   5.20000000e+00  -4.20000000e+01  -7.80000000e+01
   -1.20000000e+01]
 [  4.33800000e+03   5.58000000e+00   5.20000000e+01  -5.40000

var:             Harvard-Oxford Cortical Structural Atlas  \
0  57% Paracingulate Gyrus, 17% Cingulate Gyrus, ...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  77% Left Cerebral Cortex , 23% Left Cerebral W...   

  Juelich Histological Atlas      MNI(X Y Z)  Max_tvalue  ROI  Voxels  
0         No label found!\n'  -8.0,44.0,18.0        5.39   84  5275.0  
-52.0 -62.0 38.0
var:             Harvard-Oxford Cortical Structural Atlas  \
0  56% Lateral Occipital Cortex, superior divisio...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  87% Left Cerebral Cortex , 12% Left Cerebral W...   

                          Juelich Histological Atlas        MNI(X Y Z)  \
0  65% GM Inferior parietal lobule Pga L, 50% GM ...  -52.0,-62.0,38.0   

   Max_tvalue  ROI  Voxels  
0        5.09   84  4157.0  
58.0 -58.0 24.0
var:             Harvard-Oxford Cortical Structural Atlas  \
0  52% Angular Gyrus, 33% Lateral Occipital Corte...   

         Harvard-Oxford Subcortical 

var:             Harvard-Oxford Cortical Structural Atlas  \
0  54% Paracingulate Gyrus, 17% Frontal Pole, 8% ...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  80% Left Cerebral Cortex , 6% Right Cerebral C...   

  Juelich Histological Atlas     MNI(X Y Z)  Max_tvalue  ROI  Voxels  
0         No label found!\n'  -2.0,54.0,2.0        5.11  154  3141.0  
46.0 10.0 -42.0
var:   Harvard-Oxford Cortical Structural Atlas  \
0                     67% Temporal Pole\n'   

         Harvard-Oxford Subcortical Structural Atlas  \
0  75% Right Cerebral Cortex , 8% Right Cerebral ...   

  Juelich Histological Atlas       MNI(X Y Z)  Max_tvalue  ROI  Voxels  
0         No label found!\n'  46.0,10.0,-42.0        4.78  154  1189.0  
38.0 -30.0 -20.0
var:             Harvard-Oxford Cortical Structural Atlas  \
0  59% Temporal Fusiform Cortex, posterior divisi...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  67% Right Cerebral Cortex , 33% Right Cerebral...   

      

var:             Harvard-Oxford Cortical Structural Atlas  \
0  39% Temporal Fusiform Cortex, posterior divisi...   

         Harvard-Oxford Subcortical Structural Atlas  \
0  89% Left Cerebral Cortex , 10% Left Cerebral W...   

  Juelich Histological Atlas         MNI(X Y Z)  Max_tvalue  ROI  Voxels  
0         No label found!\n'  -42.0,-44.0,-24.0        4.91  196   163.0  
ROI Number  210
[[  2.62000000e+03   7.38000000e+00   4.80000000e+01  -4.80000000e+01
   -1.80000000e+01]
 [  2.50200000e+03   5.25000000e+00  -3.60000000e+01  -4.40000000e+01
   -2.60000000e+01]
 [  4.64000000e+02   4.77000000e+00   3.00000000e+01  -2.00000000e+00
    6.00000000e+01]
 [  3.66000000e+02   4.39000000e+00  -1.80000000e+01  -7.40000000e+01
    6.00000000e+01]
 [  3.07000000e+02   5.39000000e+00   1.80000000e+01  -7.20000000e+01
    6.00000000e+01]
 [  1.23000000e+02   3.67000000e+00  -3.00000000e+01  -1.80000000e+01
    7.20000000e+01]]
48.0 -48.0 -18.0
var:             Harvard-Oxford Cortical Stru

In [ ]:
# table.to_csv(name_csv)

In [ ]:
name_csv

In [ ]:
# columns = [['ROI', 'Voxels', 'Max_tvalue', 'MNI(X Y Z)', 'Harvard-Oxford Cortical Structural Atlas', 'Harvard-Oxford Subcortical Structural Atlas', 'Juelich Histological Atlas']]
# Index(columns)

In [ ]:
table

In [ ]:
lst_hypo

In [ ]:
df = pd.read_table( 'clustersHypoConnected/tValNegThrRoi0084_cluster.txt')

In [ ]:
df_selected = df.loc[(df['Voxels'] > 500)]
df_peak_tval_coordinates = df_selected.as_matrix(['Voxels','MAX','MAX X (mm)' ,'MAX Y (mm)','MAX Z (mm)'])

In [ ]:
df_peak_tval_coordinates

In [ ]:
table = []
for row in df_peak_tval_coordinates:
    print(row[2],row[3],row[4])
    coord = str(row[2])+ ','+ str(row[3]) + ',' + str(row[4])
    p1 = subprocess.Popen(["atlasquery", "-a", "Harvard-Oxford Cortical Structural Atlas", "-c", coord ], stdout=subprocess.PIPE)
    p2 = subprocess.Popen(["atlasquery", "-a", "Harvard-Oxford Subcortical Structural Atlas", "-c", coord], stdout=subprocess.PIPE)
    p3 = subprocess.Popen(["atlasquery", "-a", "Juelich Histological Atlas", "-c",  coord], stdout=subprocess.PIPE)
    
    out1 = p1.communicate()
    out2 = p2.communicate()
    out3 = p3.communicate()

    var = [row[0], row[1], str(out1[0]).split('>')[1], str(out1[0]).split('>')[3], \
          str(out2[0]).split('>')[1], str(out2[0]).split('>')[3], \
          str(out3[0]).split('>')[1], str(out3[0]).split('>')[3]]
    table.append(var)

In [ ]:
table

In [ ]:
str(-3.00000000e+01)

In [ ]:
str((30,30,-3.00000000e+01))

In [ ]:
str(30.0)+','+str(60.0)

In [5]:
!cat clustersHypoConnected/tValNegThrRoi0010_cluster.txt 

Cluster Index	Voxels	MAX	MAX X (mm)	MAX Y (mm)	MAX Z (mm)	COG X (mm)	COG Y (mm)	COG Z (mm)
17	563	5.05	-60	-42	-8	-61.4	-38	-8.13
16	192	5.04	46	8	-44	45	9.9	-41.5
15	46	4.09	-40	24	-20	-41.5	22.4	-21.9
14	9	4.17	6	-42	2	7.08	-42.7	1.78
13	6	3.83	36	-42	-26	36.3	-43.7	-25
12	6	3.81	16	64	26	15.7	64.3	25.6
11	5	3.83	56	-60	24	57.2	-59.2	23.6
10	4	3.72	-36	-66	-20	-37	-65	-20
9	4	3.89	-48	24	-14	-48.5	25.5	-14
8	2	3.73	-44	0	58	-45	0	57
7	2	3.65	-44	-72	46	-44	-71	46
6	2	3.53	-48	-10	-44	-49	-10	-44
5	2	3.81	-6	36	54	-6	36	55
4	1	3.57	-50	30	-16	-50	30	-16
3	1	3.58	32	-46	-22	32	-46	-22
2	1	3.69	-48	-68	46	-48	-68	46
1	1	3.78	-46	6	54	-46	6	54
